<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #5: TF-IDF
`Fecha de entrega: Septiembre 19, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances

# Punto 1: Pre-Procesamiento

- `[9 pts]` Leer el archivo `Princesas.csv` usando `pandas`
- `[9 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [2]:
data = pd.read_csv("archivos/Princesas.csv")
data

,Princesa,Personalidad
0,Blancanieves,Blancanieves es una princesa de noble cuna que...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ..."
3,Bella,Bella es una muchacha que vive en la campiña f...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince..."
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil..."
6,Mulan,Mulan es atípica a los anteriores papeles feme...
7,Tiana,Es una joven camarera que sueña con ser dueña ...
8,Mérida,Mérida llama la atención por su característico...
9,Moana,"Moana, una joven de 16 años de edad, hija únic..."


In [3]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords_sp]
    texto = " ".join([spanishStemmer.stem(palabra) for palabra in texto])
    return texto

In [4]:
data['pre-procesado'] = data['Personalidad'].apply(lambda texto: pre_procesado(texto))

In [5]:
data

,Princesa,Personalidad,pre-procesado
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campiñ frances padr inventor ...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequeñ silenci bas figur...
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer sueñ ser dueñ propi restaur algun ...
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov años edad hij unic sucesor import jef...


# Punto 2: TF-IDF

- `[16 pts]` Crear la matriz TF-IDF

In [100]:
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(data['pre-procesado'].values)

tfidf_matrix = pd.DataFrame(data=tfidf.toarray(), columns=tfidf_vect.get_feature_names())

tfidf_matrix = tfidf_matrix.T.round(3)
tfidf_matrix.columns = data.Princesa.values

tfidf_matrix

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
abrum,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
abuel,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.102
acuerd,0.000,0.0,0.056,0.000,0.000,0.000,0.0,0.000,0.08,0.000
ademas,0.000,0.0,0.000,0.000,0.000,0.099,0.0,0.000,0.00,0.000
afortun,0.087,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...
viv,0.000,0.0,0.000,0.074,0.000,0.000,0.0,0.000,0.00,0.087
volunt,0.000,0.0,0.000,0.000,0.161,0.000,0.0,0.000,0.00,0.000
volv,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
vudu,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.122,0.00,0.000


# Punto 3: Distancia del coseno
- `[12 pts]` Calcular la distancia del coseno entre cada una de las princesas
- `[2 pts]` ¿Cuáles son las princesas más parecidas?
- `[2 pts]` ¿Cuáles son las princesas más diferentes?

In [54]:
dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos = dist_cos.round(6)
dist_cos

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Blancanieves,0.000000,0.846420,0.842223,0.918112,0.935103,0.922165,0.884147,0.960082,0.985136,0.951298
Cenicienta,0.846420,0.000000,0.858412,0.947243,0.959209,0.960479,0.932663,0.918639,0.970839,0.974925
Aurora,0.842223,0.858412,0.000000,0.832503,0.937195,0.868994,0.919500,0.939231,0.940723,0.965930
Bella,0.918112,0.947243,0.832503,0.000000,0.888219,0.890513,0.882169,0.973338,0.943607,0.923546
Jasmín,0.935103,0.959209,0.937195,0.888219,0.000000,0.942753,0.935198,0.937868,0.972505,0.968929
Pocahontas,0.922165,0.960479,0.868994,0.890513,0.942753,0.000000,0.888844,0.968918,0.979646,0.936091
Mulan,0.884147,0.932663,0.919500,0.882169,0.935198,0.888844,0.000000,0.991170,0.987267,0.945844
Tiana,0.960082,0.918639,0.939231,0.973338,0.937868,0.968918,0.991170,0.000000,0.979275,0.971443
Mérida,0.985136,0.970839,0.940723,0.943607,0.972505,0.979646,0.987267,0.979275,0.000000,0.971818
Moana,0.951298,0.974925,0.965930,0.923546,0.968929,0.936091,0.945844,0.971443,0.971818,0.000000


# ¿Cuáles son las princesas más parecidas?

In [109]:
np.fill_diagonal(dist_cos.values, 999999)
resultado = round(dist_cos.min().min(),6)
resultado

0.832503

In [111]:
dist_cos.where(dist_cos==resultado).dropna(how='all').dropna(axis=1)

""
Aurora
Bella


Bella & Aurora

In [22]:
print(data[data['Princesa']=='Bella'].Personalidad.values[0])

Bella es una muchacha que vive en la campiña francesa con su padre, un inventor. Ella es una hermosa campesina que le gusta leer y seguir las aventuras de su propia imaginación. Ella no es tímida y no tiene miedo de decir lo que piensa, sobre todo en situaciones difíciles, aunque puede ser un poco vacilante cuando está nerviosa. Ella es muy compasiva, y cuida de los necesitados. Ella puede ser terca y siempre trata de proteger a la gente que realmente ama. Ella también es muy paciente con los demás.
Una soñadora y una verdadera amante de la fantasía y la aventura, Bella es una astuta inconformista para su tiempo en muchos aspectos. El más pronunciado es su amor por los libros, entusiasmo y conocimiento, lo que la gente del pueblo ve extraño para una mujer. Gastón le dice en un punto, "No es bueno que la mujer lea. Eso la hará... pensar...". Bella también no se preocupa por las apariencias, si pertenecen a la misma o cualquier otra persona. Esto es muy irónico, teniendo en cuenta que es

In [23]:
print(data[data['Princesa']=='Aurora'].Personalidad.values[0])

La Princesa Aurora, la Bella Durmiente, es la hija única de la Reina Flor y el Rey Estéfano. La Princesa Aurora, la Bella Durmiente, es retratada como una chica amable, juguetona, tímida, gentil, y bastante ingenua, que ama a los animales. Su rasgo de personalidad más destacado es su pasión por el amor y es vista como una romántica empedernida. Las hadas buenas del reino la han bendecido con la belleza y el don del canto. Ella se siente sola la mayor parte de la película, ya que está aislada del castillo de su padre dónde ha nacido en una cabaña en el bosque llamada la Cabaña del Leñador con sus tres hadas buenas y madrinas Flora, Fauna y Primavera.
En La Bella Durmiente, cuando llega de recoger las fresas, el hada Flora revela su nombre largo: Rosabelle.
Amante de las cosas simples de la vida, Aurora a menudo se pregunta por qué sus tías la tratan como a una niña. Ella desea conocer gente nueva, conocer nuevos lugares y tomar sus propias decisiones.
Pero aun así, ya que es joven e ing

# ¿Cuáles son las princesas más diferentes?

In [112]:
np.fill_diagonal(dist_cos.values, -999999)
round(dist_cos.max().max(),6)

0.99117

In [113]:
dist_cos.where(dist_cos==round(dist_cos.max().max(),6)).dropna(how='all').dropna(axis=1)

""
Mulan
Tiana


Tiana & Mulan

In [114]:
print(data[data['Princesa']=='Tiana'].Personalidad.values[0])

Es una joven camarera que sueña con ser dueña de su propio restaurante algún día. Durante una noche, ella se encuentra con el Príncipe Naveen, quien ha sido convertido en una rana por el Doctor Facilier, un malvado brujo vudú, y creyendo que ésta es una princesa debido que iba vestida como tal con un vestido de su amiga ya que se había manchado el suyo de camarera que usaba, le dice que si le besa se convertirá de nuevo en humano. Sin embargo, tras darle un beso, es Tiana quien se convierte también en rana, igual que el Príncipe Naveen. Los dos, convertidos en ranas, van en busca de alguien que rompa el hechizo.



In [115]:
print(data[data['Princesa']=='Mulan'].Personalidad.values[0])

Mulan es atípica a los anteriores papeles femeninos en las películas Disney. Es una joven bella, valiente, autosuficiente, y no se centra en el matrimonio. Asimismo, no se ajusta a las expectativas de una muchacha china de la época: elegante, discreta, y tranquila. En lugar de eso, es torpe, infantil, franca e independiente. Al comienzo de la primera película, presenta inseguridad y odio a sí misma, pensando que nunca va a traer honor a su familia. Como sigue la historia, empieza a ganar confianza en sí misma y, después de salvar a China, por fin se ve a sí misma como la persona que ella siempre quiso ser y el honor que ha traído a su familia.
Su coraje, inteligencia, determinación y extraordinario ingenio la ayuda a través de sus aventuras, mientras mantiene su respeto por los demás.



# Ustedes

In [90]:
dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos = dist_cos.round(6)
dist_cos

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Blancanieves,0.000000,0.846420,0.842223,0.918112,0.935103,0.922165,0.884147,0.960082,0.985136,0.951298
Cenicienta,0.846420,0.000000,0.858412,0.947243,0.959209,0.960479,0.932663,0.918639,0.970839,0.974925
Aurora,0.842223,0.858412,0.000000,0.832503,0.937195,0.868994,0.919500,0.939231,0.940723,0.965930
Bella,0.918112,0.947243,0.832503,0.000000,0.888219,0.890513,0.882169,0.973338,0.943607,0.923546
Jasmín,0.935103,0.959209,0.937195,0.888219,0.000000,0.942753,0.935198,0.937868,0.972505,0.968929
Pocahontas,0.922165,0.960479,0.868994,0.890513,0.942753,0.000000,0.888844,0.968918,0.979646,0.936091
Mulan,0.884147,0.932663,0.919500,0.882169,0.935198,0.888844,0.000000,0.991170,0.987267,0.945844
Tiana,0.960082,0.918639,0.939231,0.973338,0.937868,0.968918,0.991170,0.000000,0.979275,0.971443
Mérida,0.985136,0.970839,0.940723,0.943607,0.972505,0.979646,0.987267,0.979275,0.000000,0.971818
Moana,0.951298,0.974925,0.965930,0.923546,0.968929,0.936091,0.945844,0.971443,0.971818,0.000000


In [116]:
def color_red(val):
    color = 'red' if val ==True else 'black'
    return f'color: {color}'
    
t = dist_cos>0.95
t.style.applymap(color_red)

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Blancanieves,False,False,False,False,False,False,False,True,True,True
Cenicienta,False,False,False,False,True,True,False,False,True,True
Aurora,False,False,False,False,False,False,False,False,False,True
Bella,False,False,False,False,False,False,False,True,False,False
Jasmín,False,True,False,False,False,False,False,False,True,True
Pocahontas,False,True,False,False,False,False,False,True,True,False
Mulan,False,False,False,False,False,False,False,True,True,False
Tiana,True,False,False,True,False,True,True,False,True,True
Mérida,True,True,False,False,True,True,True,True,False,True
Moana,True,True,True,False,True,False,False,True,True,False


In [117]:
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

def highlight_min(s):
    min_val = min(sorted(s)[1:])
    is_max = s == min_val
    return ['background-color: green' if v else '' for v in is_max]

dist_cos.style.apply(highlight_max).apply(highlight_min)

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Blancanieves,-999999,0.84642,0.842223,0.918112,0.935103,0.922165,0.884147,0.960082,0.985136,0.951298
Cenicienta,0.84642,-999999,0.858412,0.947243,0.959209,0.960479,0.932663,0.918639,0.970839,0.974925
Aurora,0.842223,0.858412,-999999,0.832503,0.937195,0.868994,0.9195,0.939231,0.940723,0.96593
Bella,0.918112,0.947243,0.832503,-999999,0.888219,0.890513,0.882169,0.973338,0.943607,0.923546
Jasmín,0.935103,0.959209,0.937195,0.888219,-999999,0.942753,0.935198,0.937868,0.972505,0.968929
Pocahontas,0.922165,0.960479,0.868994,0.890513,0.942753,-999999,0.888844,0.968918,0.979646,0.936091
Mulan,0.884147,0.932663,0.9195,0.882169,0.935198,0.888844,-999999,0.99117,0.987267,0.945844
Tiana,0.960082,0.918639,0.939231,0.973338,0.937868,0.968918,0.99117,-999999,0.979275,0.971443
Mérida,0.985136,0.970839,0.940723,0.943607,0.972505,0.979646,0.987267,0.979275,-999999,0.971818
Moana,0.951298,0.974925,0.96593,0.923546,0.968929,0.936091,0.945844,0.971443,0.971818,-999999
